In [ ]:
#!unzip Test.zip

: 

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

: 

In [17]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [ ]:
len(pgm_kaggle_files)

In [ ]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

In [ ]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

In [21]:
import joblib

In [22]:
# Aquí debe utilizar su modelo
model = joblib.load('models/tree_model.joblib')

In [ ]:
from skimage import feature

# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = []

suffix = '.pgm'
dir = os.listdir('Test/')
pgm_files = [filename for filename in dir if filename.endswith(suffix)]

for filename in tqdm(pgm_files):
    path = 'Test/' + filename
    with open (path, 'rb') as pgmf:
        image = plt.imread(pgmf)
        X_kag.append(image)
        
X = np.array([feature.hog(im) for im in tqdm(X_kag, desc='Construyendo X') ])


In [ ]:
from sklearn.decomposition import PCA 

pca = PCA(n_components=500)
X_pca_kag = pca.fit_transform(X)

# Predicción
y_kag = model.predict(X_pca_kag)

In [11]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [ ]:
kaggle_hat.head()

In [ ]:
from datetime import datetime

fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M")

print(fecha_hora)

In [14]:
# Guardamos en un .csv para subir a kaggle
submission_name = f'tree_{fecha_hora}'
kaggle_hat.to_csv(submission_name + '.csv', index=False)